<a href="https://colab.research.google.com/github/Jenniferduong20012004/JennyLearningAI/blob/main/LLMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#LLMS
#computer vision: dữ liệu đầu vào là ảnh và video
#natural language processing: văn bản text
#time series: dự báo thời tiết, chứng khoáng